<h1> Sentiment Classification Using DistilBert and Pytorch</h1>

In this notebook, we will see a great overview of how to classify sentences using DistilBert pretrained by HuggingFace and a fine-tuned Neural Network to classify the sentiment.




In [31]:
!pip install transformers

In [32]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers
import warnings
warnings.filterwarnings('ignore')

In [33]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv',delimiter='\t',header=None)

In [ ]:
df = df[:2000]

In [ ]:
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer,'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


In [ ]:
tokenized = df[0].apply(lambda x: tokenizer.encode(x,add_special_tokens=True))

In [ ]:
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

In [ ]:
attention_mask = np.where(padded != 0 , 1, 0)
attention_mask.shape

In [ ]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids,attention_mask=attention_mask) 

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = df[1]

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
parameters = {'C' : np.linspace(0.0001,100,20)}
grid_search = GridSearchCV(LogisticRegression(),parameters)
grid_search.fit(train_features,train_labels)

print(grid_search.best_params_)
print(grid_search.best_score_)

In [ ]:
lr_clf = LogisticRegression(C= 5.2)
lr_clf.fit(train_features,train_labels)


In [ ]:
lr_clf.score(test_features,test_labels)

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf,train_features,train_labels)
scores.mean()

<h1> Neural Networks </h1>

In [30]:
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
import torch

In [85]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,df,out,max_len=96):
        
        self.df = df
        self.out = out
        self.max_len = max_len
        self.tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        
    def __getitem__(self,index):
        data = {}
        row = self.df.iloc[index]
        ids,masks,labels = self.get_input_data(row)
        data['ids'] = torch.tensor(ids)
        data['masks'] = masks
        data['labels'] = torch.tensor(self.out.iloc[index].values[0],dtype=torch.float32)
        return data
    
    def __len__(self):
        return len(self.df)
    
    def get_input_data(self,row):
        ids = self.tokenizer.encode(row[0],add_special_tokens=True)
        pad_len = self.max_len - len(ids)
        if pad_len > 0 :
            ids += [0]*pad_len
        ids = torch.tensor(ids)    
        masks = torch.where(ids != 0 , torch.tensor(1),torch.tensor(0))
        return ids,masks,self.out.iloc[0].values
    
        

In [188]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv',delimiter='\t',header=None)
train_set,val_set = train_test_split(df,test_size = 0.2)
train_loader = torch.utils.data.DataLoader(Dataset(pd.DataFrame(train_set[0]),pd.DataFrame(train_set[1]),max_len=80),batch_size = 32, shuffle = True, num_workers = 2)
val_loader = torch.utils.data.DataLoader(Dataset(pd.DataFrame(val_set[0]),pd.DataFrame(val_set[1]),max_len=80),batch_size = 32,num_workers = 2)

In [102]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc.item()

In [135]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        config = transformers.DistilBertConfig.from_pretrained('distilbert-base-uncased')
        self.distilBert = transformers.DistilBertModel.from_pretrained('distilbert-base-uncased',config=config)
        self.fc0 = nn.Linear(768,512)
        self.d0 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(512,256)
        self.d1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256,1)
        nn.init.normal_(self.fc0.weight,std= 0.1)
        nn.init.normal_(self.fc0.bias ,0.)
        nn.init.normal_(self.fc1.weight,std =0.1)
        nn.init.normal_(self.fc1.bias, 0.)
        nn.init.normal_(self.fc2.weight,std=0.1)
        nn.init.normal_(self.fc2.bias , 0.)

        
    def forward(self,input_ids,attention_mask):
        hid= self.distilBert(input_ids,attention_mask = attention_mask)
        hid= hid[0][:,0]
        x = self.fc0(hid)
        x = F.relu(x)
        x = self.d0(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.d1(x)
        return self.fc2(x)


In [191]:
criterion = nn.BCEWithLogitsLoss(reduction='mean').to('cuda')
model = Model().to('cuda')
for params in model.distilBert.parameters():
    params.require_grad = False
    params._trainable = False
optimizer = torch.optim.AdamW(model.parameters(),lr=2e-5)

In [192]:
from tqdm import tqdm
epochs = 5

In [193]:

for epoch in range(epochs):
        
        epoch_loss = 0
        val_loss = 0
        correct = 0
        accuracy = 0

        for data in train_loader:
            ids = data['ids'].cuda()
            masks = data['masks'].cuda()
            labels = data['labels'].cuda()
            labels = labels.unsqueeze(1)
            model.train()
            optimizer.zero_grad()
            outputs = model(ids,masks)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()
           
            epoch_loss += loss.item()
            
        print(f'Train Epoch {epoch} : Loss {epoch_loss/len(train_loader)}')
        print("Train Accuracy : ",binary_acc(outputs,labels))
        model.eval()
        correct = 0
        for data in val_loader:
            ids = data['ids'].cuda()
            masks = data['masks'].cuda()
            labels = data['labels'].cuda()
            outputs = model(ids,masks)
            labels = labels.unsqueeze(1)
            loss = criterion(outputs,labels)
            val_loss += loss.item()
            
        print(f'Val Epoch {epoch} : Loss {val_loss/len(val_loader)}')
        print("Val Accuracy : ",binary_acc(outputs,labels))
        if binary_acc(outputs,labels) >90:
            break


Train Epoch 0 : Loss 1.6381001019477843
Train Accuracy :  38.0
Val Epoch 0 : Loss 0.551078716149697
Val Accuracy :  69.0
Train Epoch 1 : Loss 0.8140286940336228
Train Accuracy :  81.0
Val Epoch 1 : Loss 0.5062685471314651
Val Accuracy :  69.0
Train Epoch 2 : Loss 0.5237828180193901
Train Accuracy :  75.0
Val Epoch 2 : Loss 0.5580048606945918
Val Accuracy :  88.0
Train Epoch 3 : Loss 0.3948425027728081
Train Accuracy :  97.0
Val Epoch 3 : Loss 0.5113603518559382
Val Accuracy :  88.0
Train Epoch 4 : Loss 0.20395209703594447
Train Accuracy :  88.0
Val Epoch 4 : Loss 0.7078863932536199
Val Accuracy :  88.0


In [178]:
def test(sentence):
    tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    ids = tokenizer.encode(sentence,add_special_tokens=True)
    padded = ids + [0]*(80 - len(ids))
    padded = torch.tensor(padded,dtype=torch.int64).unsqueeze(0)
    masks = torch.where(padded != 0 , torch.tensor(1), torch.tensor(0)).cuda()
    padded = padded.cuda()
    model.eval()
    output = model(padded,masks)
    return torch.round(F.sigmoid(output)).item()

In [183]:
test("I am feeling warm by this content.")

1.0